This jupyter notebook is to show function `babs-init`.

In [1]:
# See `README_howtorun_show_babs_init_ipynb.md` for how to run this file!

# Clone out:
#   datalad clone ria+file:///path/to/<container>/output_ria#~data singleORmulti-ses_<container>_outputs
# If you want to change `project_root`'s foldername, also change `output_ria/alias/data` symlink:
    # cd output_ria/alias
    # ln -sf /new/full/path/to/output_ria/xxx/xxx-xxx-xxx-xxx data

# flake8: noqa

# set up how Jupyter notebook behaves:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'   # print all outputs
 
# import sys
import os
import os.path as op
# sys.path.append( os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(os.path.abspath('')))), "babs"  ))

# from babs.core_functions import babs_init
# from babs.cli import *


There are several arguments when calling `babs-init`:

In [4]:
# This notebook only accepts zipped input ds (toybidsapp with --zipped; xcpd)
# ++++++++++++++++++++++++++++++++++
flag_instance = "fmriprep_ingressed_fs"
type_session = "multi-ses"
is_real_data = True

flag_where = "cubic"     # "cubic" or "local"
# ++++++++++++++++++++++++++++++++++
where_notebooks = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper/babs/notebooks"

# where:
if flag_where == "cubic":
    where_root = "/cbica/projects/BABS"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
elif flag_where == "local":
    where_root = "/Users/chenyzh/Desktop/Research/Satterthwaite_Lab/datalad_wrapper"
    where_project = op.join(where_root, "data")
    where_notebooks = op.join(where_root, "babs/notebooks")
else:
    raise Exception("not valid `flag_where`!")

# Input dataset:
if is_real_data is False:
    if type_session == "multi-ses":
        input_ds = op.join(where_project, "k9zw2")
    elif type_session == "single-ses":
        input_ds = op.join(where_project, "2jvub")
else:           # real data:
    if type_session == "multi-ses":
        # input_ds = "/cbica/projects/RBC/chenying_practice/data_for_babs/NKI/data_hashedID_bids"
        input_ds = "/cbica/projects/BABS/data/testdata_NKI/fmriprep_multises_outputs"
        # ^^ clone from output_ria in `fmriprep-multises` in RBC cubic project
        # note: use fmriprep full outputs as input to test; 
        #   current set up of `fmriprep_ingressed_fs` should ignore existing `_fmriprep.zip`

        input_ds_bids = "/cbica/projects/BABS/data/testdata_NKI/data_hashedID_bids"
        # ^^ raw BIDS data, used by `fmriprep_ingressed_fs`
    elif type_session == "single-ses":
        raise Exception("not supported yet!")

# Based on which BIDS App:
if flag_instance == "zipped_toybidsapp":
    input_cli = [["BIDS", input_ds]]
    project_name = "test_babs_" + type_session + "_" + flag_instance
    bidsapp = "toybidsapp"
    container_name = bidsapp + "-0-0-5"
    input_ds_name = "fmriprep"

elif flag_instance == "xcpd":
    project_name = "test_babs_" + type_session + "_" + flag_instance
    bidsapp = "xcpd"
    container_name = bidsapp + "-0-3-0"
    input_ds_name = "fmriprep"
    
elif flag_instance == "fmriprep_ingressed_fs":
    project_name = "test_babs_" + type_session + "_" + "fpfsin"
    bidsapp = "fmriprep"
    container_name = bidsapp + "-20-2-3"
    # zipped input ds's name:
    input_ds_name = "freesurfer"
    
else:
    raise Exception("`flag_instance` is not toybidsapp or xcpd!")

# Container:
container_ds = op.join(where_project, bidsapp + "-container")  
if flag_where == "local":
    container_ds += "-docker"   # add "docker" at the end
container_config_yaml_file = op.join(where_notebooks, "example_container_" + flag_instance + ".yaml")


# list_sub_file = op.join(where_notebooks, "initial_sub_list_" + type_session + ".csv")
list_sub_file = None

Let's pass these arguments into `babs-init` CLI:

In [5]:
cmd = "babs-init \\\n"
cmd += "\t" + "--where_project " + where_project + " \\\n"
cmd += "\t" + "--project_name " + project_name + " \\\n"
if flag_instance == "fmriprep_ingressed_fs":
    cmd += "\t" + "--input " + "BIDS" + " " + input_ds_bids + " \\\n"
cmd += "\t" + "--input " + input_ds_name + " " + input_ds + " \\\n"
if list_sub_file is not None:
    cmd += "\t" + "--list_sub_file " + list_sub_file + " \\\n"
cmd += "\t" + "--container_ds " + container_ds + " \\\n"
cmd += "\t" + "--container_name " + container_name + " \\\n"
cmd += "\t" + "--container_config_yaml_file " + container_config_yaml_file + " \\\n"
cmd += "\t" + "--type_session " + type_session + " \\\n"
cmd += "\t" + "--type_system " + "sge"

print("The command to execute:")
print(cmd)


The command to execute:
babs-init \
	--where_project /cbica/projects/BABS/data \
	--project_name test_babs_multi-ses_fpfsin \
	--input BIDS /cbica/projects/BABS/data/testdata_NKI/data_hashedID_bids \
	--input freesurfer /cbica/projects/BABS/data/testdata_NKI/fmriprep_multises_outputs \
	--container_ds /cbica/projects/BABS/data/fmriprep-container \
	--container_name fmriprep-20-2-3 \
	--container_config_yaml_file /cbica/projects/BABS/babs/notebooks/example_container_fmriprep_ingressed_fs.yaml \
	--type_session multi-ses \
	--type_system sge


In [ ]:
!($cmd)